# DT Assignment1

# Data Loading

In [22]:
import pandas as pd 
import numpy as np

In [23]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [24]:
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [25]:
## 처음에 gini 계수를 잘못이해하고 데이터와 targeta 변수에 대한 모든 변수의 지니계수를 구하고 그걸 이용하는 것으로 착각함..ㅠ

def get_gini(df, label):
    
    gini_list2 = []

    for i in range(len(df.columns)-1):
        gini_list1 = []
        gini_value = 0
        print(f'\n[{df.columns[i]} 변수진입]')
        n = len(df) # 전체 데이터 개수
#         print('전체데이터 : ', n)
        cn = len(np.unique(df.values[:,i])) # 변수의 class 개수
        ln = len(df[label].unique()) # label 변수의 class 개수
#         print('변수 개수 :', cn)
        
        for q in range(cn):
            print(np.unique(df.values[:,i])[q], '진입')
            value = 0
            value2 = 0
            raw_gini = 0
            raw_gini2 = 0
            
            cln = sum(df.values[:,i] == np.unique(df.values[:,i])[q]) # 한 변수가 가지고 있는 첫번째 class 진입
#             print('분자값:',cln)
            
            for z in range(ln):
                lln = sum(df.loc[df.values[:,i] == np.unique(df.values[:,i])[q], label ] == df[label].unique()[z])
#                 print(lln)
                if lln == 0:
                    value += 0
                else:
                    value += (lln / cln)**2
#             print('value:',value)
            raw_gini = (cln / n) * (1 - value)
            
#             print('분자값:', n-cln)
            for t in range(ln):
                lln2 = sum(df.loc[df.values[:,i] != np.unique(df.values[:,i])[q], label ] == df[label].unique()[t])
#                 print(lln2)
                if lln2 == 0:
                    value2 += 0
                else:
                    value2 += (lln2 / (n - cln))**2
#             print('value2:', value2)
            raw_gini2 = ((n - cln) / n) * (1 - value2)
            gini_value = raw_gini + raw_gini2
            print(gini_value)
        
            gini_list1.append(gini_value)
        gini_raw = min(gini_list1)
        
        gini_list2.append(gini_raw)
        
    p = 0
    for row in gini_list2:
        print(df.columns[p],":",row)
        p += 1
    
    gini = min(gini_list2)    
    
    return gini

In [26]:
get_gini(pd_data,'class_buys_computer')


[age 변수진입]
middle_aged 진입
0.35714285714285715
senior 진입
0.45714285714285713
youth 진입
0.3936507936507937

[income 변수진입]
high 진입
0.4428571428571429
low 진입
0.45
medium 진입
0.4583333333333333

[student 변수진입]
no 진입
0.3673469387755103
yes 진입
0.3673469387755103

[credit_rating 변수진입]
excellent 진입
0.42857142857142855
fair 진입
0.42857142857142855
age : 0.35714285714285715
income : 0.4428571428571429
student : 0.3673469387755103
credit_rating : 0.42857142857142855


0.35714285714285715

In [27]:
def get_gini(df, label):
    n = len(df) # 전체 데이터 개수
    cn = len(df[label].unique()) # 타켓 변수의 class 개수 (yes/no 일때는 2개 )
    value = 0 
    for i in range(cn):
        ln = sum(df[label] == df[label].unique()[i]) # 각 클래스와 일치하는 개수
        if ln == 0: # 분모가 0 일때는 0 으로 수렴
            value += 0
        else:
            value += (ln / n)**2 # 빼줄 값들
    
    gini = (1 - value) # 공식
        
    return gini

In [64]:
def get_gini(df, label):
    D_len = df[label].count() # 데이터 전체 길이
    # 각 클래스별 Count를 담은 Generator 생성
    count_arr = (value for key, value in df[label].value_counts().items())
    # reduce를 이용해 초기값 1에서 각 클래스 (count / D_len)^2 빼기
    return reduce(lambda x, y: x - (y/D_len)**2 ,count_arr,1)


In [28]:
get_gini(pd_data, 'income')

0.653061224489796

In [29]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

In [30]:
get_gini(pd_data, 'student')

0.5

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [31]:
from itertools import combinations # 변수의 모든 클래시 조합을 얻기 위해 itertools 불러오기

In [32]:
def get_binary_split(df, attribute):
    
    lev = df[attribute].unique() # 특정 칼럼의 class 개수
    data = []
    if len(lev) == 1:
        re = lev
    else:
        for i in range(len(lev)-1): 
            elem = combinations(lev,i+1) # 모든 조합 생성 (0번째 조합 생성 X)
            temp = list(elem)
            for q in range(len(temp)): # 생성된 조합을 각각 택함
                temp2 = list(temp[q])
                data.append(temp2)
                re = data
    return re

In [33]:
get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

In [34]:
def get_binary_split(df, attribute):
    
    lev = df[attribute].unique() # 특정 칼럼의 class 개수
    data = []
    for i in range(len(lev)-1): 
        elem = combinations(lev,i+1) # 모든 조합 생성 (0번째 조합 생성 X)
        temp = list(elem)
        for q in range(len(temp)): # 생성된 조합을 각각 택함
            temp2 = temp[q]
            data.append(temp2)
            re= data
    return re

In [35]:
get_binary_split(pd_data,"age")

[('youth',),
 ('middle_aged',),
 ('senior',),
 ('youth', 'middle_aged'),
 ('youth', 'senior'),
 ('middle_aged', 'senior')]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.
- 결과로 나온 Dictionary의 Key 값은 해당 class 들로 이루어진 tuple 형태로 들어가 있습니다.

In [36]:
len(pd_data['age'])

14

In [37]:
def get_attribute_gini_index(df, attribute, label):
    index_list = []
    c_class = get_binary_split(df, attribute) # 변수의 이진분류 집합
    for c in c_class: 
        n = len(df) # 총 데이터 개수
        temp = pd.DataFrame()
        for i in range(len(c)): 
            if len(c) == 1: # 집합의 원소가 1개일 경우
                temp = df.loc[df[attribute] == c[i]]
                temp2 = df.loc[df[attribute] != c[i]]

            else: # 집합의 원소가 1개 이상일 경우

                temp_df = df.loc[df[attribute] == c[i]]
                temp_df2 = df.loc[df[attribute] != c[i]]
                
                temp = pd.concat((temp, temp_df), axis = 0) # 원소에 해당되는 데이터프레임 계속해서 이어붙이기

                if i == 0:
                    temp2 = temp_df2.copy()
                if i > 0:
                    temp2 = pd.merge(temp2, temp_df2) # 원소에 해당되지 않는 데이터프레임 합치기 (2번째부터)

        gini = get_gini(temp, label)
        gini2 = get_gini(temp2, label)
        value = (len(temp) / n ) * gini
        value2 = (len(temp2) / n) * gini2
        index = value + value2 # 한 집합의 gini_index 값
        
        index_list.append(index)
        
    c_df = pd.DataFrame(c_class) # 리스트 값을 딕셔너리 값 하나에 넣기 위해서 원소를 합치는 과정 (매우 비효율적인것같음..ㅠ, 효율적인 방법을 찾지 못함)
    c_df = c_df.fillna('')
    c_list = []
    for i in range(len(c_df)):
        value = ', '.join(c_df.loc[i])
        c_list.append(value)
        
    result = dict(zip(c_list, index_list)) # 두 개의 리스트를 딕셔너리 값으로 변환
    return result

In [38]:
def get_gini(df, label):
    D_len = df[label].count() # 데이터 전체 길이
    # 각 클래스별 Count를 담은 Generator 생성
    count_arr = (value for key, value in df[label].value_counts().items())
    # reduce를 이용해 초기값 1에서 각 클래스 (count / D_len)^2 빼기
    return reduce(lambda x, y: x - (y/D_len)**2 ,count_arr,1)

In [39]:
def get_binary_split(df, attribute):
    attr_unique = df[attribute].unique()
    # 이중 For loop List Comprehension
    result = [
            list(item) 
            for i in range(1, len(attr_unique)) # 1부터 변수의 클래스 갯수-1 까지 Iteration
            for item in itertools.combinations(attr_unique, i) # i를 길이로 하는 조합 생성
        ]
    return result

In [7]:
get_gini(pd_data, 'class_buys_computer')

0.4591836734693877

In [17]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth, ': 0.3936507936507937,
 'middle_aged, ': 0.35714285714285715,
 'senior, ': 0.45714285714285713,
 'youth, middle_aged': 0.45714285714285713,
 'youth, senior': 0.35714285714285715,
 'middle_aged, senior': 0.3936507936507937}

In [245]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth, ': 0.3936507936507937,
 'middle_aged, ': 0.35714285714285715,
 'senior, ': 0.45714285714285713,
 'youth, middle_aged': 0.45714285714285713,
 'youth, senior': 0.35714285714285715,
 'middle_aged, senior': 0.3936507936507937}

In [44]:
get_attribute_gini_index(pd_data, "student", "class_buys_computer")

{'no': 0.3673469387755103, 'yes': 0.3673469387755103}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

결과를 확인해보도록 하겠습니다.

In [18]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - middle_aged,  : 0.35714285714285715


# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [248]:
##문제1 답안
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [83]:
##문제2 답안

def get_gini_index(df, label):
    attributes = df.columns[:df.shape[1]-1]
    for attribute in attributes:
        print(attribute,'변수')
        index_list = []
        c_class = get_binary_split(df, attribute) # 변수의 이진분류 집합
        for c in c_class: 
            n = len(df) # 총 데이터 개수
            temp = pd.DataFrame()
            for i in range(len(c)): 
                if len(c) == 1: # 집합의 원소가 1개일 경우
                    temp = df.loc[df[attribute] == c[i]]
                    temp2 = df.loc[df[attribute] != c[i]]

                else: # 집합의 원소가 1개 이상일 경우

                    temp_df = df.loc[df[attribute] == c[i]]
                    temp_df2 = df.loc[df[attribute] != c[i]]

                    temp = pd.concat((temp, temp_df), axis = 0) # 원소에 해당되는 데이터프레임 계속해서 이어붙이기

                    if i == 0:
                        temp2 = temp_df2.copy()
                    if i > 0:
                        temp2 = pd.merge(temp2, temp_df2) # 원소에 해당되지 않는 데이터프레임 합치기 (2번째부터)

            gini = get_gini(temp, label)
            gini2 = get_gini(temp2, label)
            value = (len(temp) / n ) * gini
            value2 = (len(temp2) / n) * gini2
            index = value + value2 # 한 집합의 gini_index 값

            index_list.append(index)

        c_df = pd.DataFrame(c_class)
        c_df = c_df.fillna('')
        c_list = []
        for i in range(len(c_df)):
            value = ', '.join(c_df.loc[i])
            c_list.append(value)

        result = dict(zip(c_list, index_list))
        
        km = min(result.keys(), key = (lambda k: result[k])) # 한 변수의 여러 index 값에서 가장 작은 값을 가지는 원소와 그 수치를 가져옴
        print(km, result[km],'\n') # 모든 변수의 가장 작은 값을 가지는 원소와 수치 출력
        
    return 


In [94]:
# gini index 가 가장 작은 경우는 age 변수의 middle_age 경우임
# 즉, 최초 split 되는 변수는 'age' 이며 기준은 'middle_aged' 라고 볼 수 있음
get_gini_index(pd_data, 'class_buys_computer')

age 변수
middle_aged,  0.35714285714285715 

income 변수
high,  0.4428571428571429 

student 변수
no 0.3673469387755103 

credit_rating 변수
fair 0.42857142857142855 



In [63]:
##문제3 답안
# age 변수에서 middle_aged 로 분류된 첫번째 데이터프레임은 모두 gini index 값이 0 값을 가짐
# 두번째 데이터프레임에서는 student 변수에서 no 가 0.31999.. 로 가장 작은 값을 가지므로 그 다음 split 기준으로 선정할 수 있음

Fst_split1 = pd_data.loc[pd_data['age'] == 'middle_aged']
Fst_split2 = pd_data.loc[pd_data['age'] != 'middle_aged']

In [96]:
get_gini_index(Fst_split1, 'class_buys_computer')

age 변수
middle_aged 0.0 

income 변수
high,  0.0 

student 변수
no 0.0 

credit_rating 변수
fair 0.0 



In [75]:
get_gini_index(Fst_split2, 'class_buys_computer')

age 변수
youth 0.48 

income 변수
high,  0.375 

student 변수
no 0.31999999999999984 

credit_rating 변수
fair 0.4166666666666667 

